
  **CHRIST (Deemed to be University), Bangalore – 560 029**  
  **Department of Statistics and Data Science**  
  **M.Sc. Statistics Practicals**  
  **MSTAT472A: Neural Networks and Deep Learning**  
  **Title: CNN Image Classification**  
  Submitted on 20th Aug, 2025.
  
  **Submitted to:**  
  Dr. Dalvin Vinoth Kumar A,  
Dept of Statistics and Data Science,  
Christ University, Bangalore Central Campus  

**Submitted by:**  
  Aditi Prasad Chandavarkar,  
2448104,  
4MSTAT



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, ResNet50, VGG16
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.resnet import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
import numpy as np
import pandas as pd


# Dataset Folder
data_dir = "/content/drive/MyDrive/CNN Image classification/FINAL RESIZED IMAGE CNN"
img_size = (128, 128)
batch_size = 16

# Rescaling and splitting the data (80-20 split used))
datagen = ImageDataGenerator(validation_split=0.2)
train_gen_base = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen_base = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

class_names = list(train_gen_base.class_indices.keys())
print("Classes:", class_names)

# Helper Function to Build Models
def build_model(base_model, preprocess_func, name):
    base_model.trainable = False

    model = models.Sequential([
        layers.Input(shape=(128, 128, 3)),
        layers.Lambda(preprocess_func),
        base_model,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(len(class_names), activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    print(f"\n{name} Summary:")
    model.summary()
    return model

# Training Pretrained Models
# MobileNetV2
mobilenet_model = build_model(MobileNetV2(weights="imagenet", include_top=False, input_shape=(128,128,3)),
                              mobilenet_preprocess,
                              "MobileNetV2")

history_mobilenet = mobilenet_model.fit(
    train_gen_base,
    epochs=10,
    validation_data=val_gen_base
)

# ResNet50
resnet_model = build_model(ResNet50(weights="imagenet", include_top=False, input_shape=(128,128,3)),
                           resnet_preprocess,
                           "ResNet50")

history_resnet = resnet_model.fit(
    train_gen_base,
    epochs=10,
    validation_data=val_gen_base
)

# VGG16
vgg_model = build_model(VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3)),
                        vgg_preprocess,
                        "VGG16")

history_vgg = vgg_model.fit(
    train_gen_base,
    epochs=10,
    validation_data=val_gen_base
)

# Predictions & Comparison
def get_predictions(model, generator, model_name):
    preds = model.predict(generator)
    y_pred = np.argmax(preds, axis=1)
    y_true = generator.classes
    acc = np.mean(y_pred == y_true) * 100
    return model_name, acc

results = []
results.append(get_predictions(mobilenet_model, val_gen_base, "MobileNetV2"))
results.append(get_predictions(resnet_model, val_gen_base, "ResNet50"))
results.append(get_predictions(vgg_model, val_gen_base, "VGG16"))

# Comparison Table for Results
results_df = pd.DataFrame(results, columns=["Model", "Validation Accuracy (%)"])
print("\nModel Comparison:\n", results_df)


Found 297 images belonging to 4 classes.
Found 73 images belonging to 4 classes.
Classes: ['.ipynb_checkpoints', 'FIST', 'PEACE', 'THUMBS UP']

MobileNetV2 Summary:


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda_3 (Lambda)               │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 20480)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     2,621,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,880,068 (18.62 MB)

 Trainable params: 2,622,084 (10.00 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lambda_5 (Lambda)               │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,763,908 (60.13 MB)

 Trainable params: 1,049,220 (4.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 71s 4s/step - accuracy: 0.4102 - loss: 12.0088 - val_accuracy: 0.7945 - val_loss: 1.4895
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 67s 4s/step - accuracy: 0.8471 - loss: 1.3927 - val_accuracy: 0.7671 - val_loss: 1.4379
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.9442 - loss: 0.4602 - val_accuracy: 0.8904 - val_loss: 0.5388
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.9343 - loss: 0.2744 - val_accuracy: 0.8356 - val_loss: 0.6048
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.9442 - loss: 0.2441 - val_accuracy: 0.9452 - val_loss: 0.2528
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.9769 - loss: 0.0481 - val_accuracy: 0.9452 - val_loss: 0.2869
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 68s 4s/step - accuracy: 0.9812 - loss: 0.1085 - val_accuracy: 0.9178 - val_loss: 0.5135
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 66s 4s/step - accuracy: 0.9728 - loss: 0.1361 - val_accuracy: 0.9178 - val_loss

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image

# Loading Dataset
data_dir = "/content/drive/MyDrive/CNN Image classification/FINAL RESIZED IMAGE CNN"
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Splitting Training and Validation set (80-20)
train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

class_labels = list(train_gen.class_indices.keys())

# Custom CNN Architecutre using Sequential Function

# Define improved custom model
custom_model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),   # stronger dropout to prevent overfitting
    layers.Dense(len(class_labels), activation='softmax')
])

# Compile
custom_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6)

# Train
history = custom_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stop, reduce_lr]
)

results.append(get_predictions(custom_model, val_gen_base, "Custom Architecture"))


Found 297 images belonging to 4 classes.
Found 73 images belonging to 4 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 7s 258ms/step - accuracy: 0.2985 - loss: 1.2736 - val_accuracy: 0.3699 - val_loss: 1.1737 - learning_rate: 5.0000e-04
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 329ms/step - accuracy: 0.3460 - loss: 1.1607 - val_accuracy: 0.3973 - val_loss: 1.1125 - learning_rate: 5.0000e-04
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 247ms/step - accuracy: 0.3682 - loss: 1.1604 - val_accuracy: 0.4795 - val_loss: 1.0869 - learning_rate: 5.0000e-04
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 252ms/step - accuracy: 0.4522 - loss: 1.0673 - val_accuracy: 0.3836 - val_loss: 1.1047 - learning_rate: 5.0000e-04
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 6s 341ms/step - accuracy: 0.4618 - loss: 1.0688 - val_accuracy: 0.4247 - val_loss: 1.0577 - learning_rate: 5.0000e-04
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 242ms/step - accuracy: 0.4664 - loss: 1.0985 - val_accuracy: 0.5616 - val_loss: 1.0301 - learning_rate: 5.0000e-04
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 249ms/step - accuracy: 

In [ ]:
# Accuracy Comparison

results_df = pd.DataFrame(results, columns=["Model", "Validation Accuracy (%)"])
print("\nModel Comparison:\n", results_df)


Model Comparison:
                  Model  Validation Accuracy (%)
0          MobileNetV2                95.890411
1             ResNet50                91.780822
2                VGG16                90.410959
3  Custom Architecture                76.712329


In [67]:
# Prediction Function
def predict_image(model, img_path, labels):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    preds = model.predict(img_array, verbose=0)
    return labels[np.argmax(preds)]

# 5. Compare Predictions
test_image_path = "/content/drive/MyDrive/CNN Image classification/THUMBS UP/IMG-20250817-WA0008.jpg"

results = {
    "Model": ["MobileNet", "ResNet", "VGG", "Custom CNN"],
    "Prediction": [
        predict_image(mobilenet_model, test_image_path, class_labels),
        predict_image(resnet_model, test_image_path, class_labels),
        predict_image(vgg_model, test_image_path, class_labels),
        predict_image(custom_model, test_image_path, class_labels)
    ]
}

results_df = pd.DataFrame(results)
print(results_df)


        Model Prediction
0   MobileNet       FIST
1      ResNet      PEACE
2         VGG      PEACE
3  Custom CNN  THUMBS UP


In [88]:
test_image_path = "/content/drive/MyDrive/CNN Image classification/PEACE/IMG-20250817-WA0000.jpg"

results = {
    "Model": ["MobileNet", "ResNet", "VGG", "Custom CNN"],
    "Prediction": [
        predict_image(mobilenet_model, test_image_path, class_labels),
        predict_image(resnet_model, test_image_path, class_labels),
        predict_image(vgg_model, test_image_path, class_labels),
        predict_image(custom_model, test_image_path, class_labels)
    ]
}

results_df = pd.DataFrame(results)
print(results_df)

        Model Prediction
0   MobileNet       FIST
1      ResNet      PEACE
2         VGG      PEACE
3  Custom CNN      PEACE


In [ ]:
test_image_path = "/content/drive/MyDrive/CNN Image classification/FIST/IMG-20250817-WA0007.jpg"
results = {
    "Model": ["MobileNet", "ResNet", "VGG", "Custom CNN"],
    "Prediction": [
        predict_image(mobilenet_model, test_image_path, class_labels),
        predict_image(resnet_model, test_image_path, class_labels),
        predict_image(vgg_model, test_image_path, class_labels),
        predict_image(custom_model, test_image_path, class_labels)
    ]
}

results_df = pd.DataFrame(results)
print(results_df)

        Model Prediction
0   MobileNet       FIST
1      ResNet       FIST
2         VGG       FIST
3  Custom CNN       FIST
